In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Defining file path
ecg_folder = "../../../Datasets/12-lead electrocardiogram database/ECGDataDenoised"
attributes_file = "../../../Datasets/12-lead electrocardiogram database/AttributesDictionary.xlsx"
diagnostics_file = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"
rhythm_names_file = "../../../Datasets/12-lead electrocardiogram database/RhythmNames.xlsx"

# Checking for missing files and stuff
for file_path in [attributes_file, diagnostics_file, rhythm_names_file]:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Required file not found: {file_path}")

# Loading metadata
attributes_df = pd.read_excel(attributes_file)
diagnostics_df = pd.read_excel(diagnostics_file)
rhythm_names_df = pd.read_excel(rhythm_names_file)

# Removing trailing spaces in acronym columns for accurate matching
rhythm_names_df['Acronym Name'] = rhythm_names_df['Acronym Name'].str.strip()

# Creating sets of valid acronyms for rhythm
valid_rhythms = set(rhythm_names_df['Acronym Name'])
print(valid_rhythms)

{'AVNRT', 'AF', 'AT', 'SVT', 'SR', 'SI', 'SB', 'AFIB', 'SAAWR', 'AVRT', 'ST'}


In [2]:
# Load ECG data
def load_ecg_data(ecg_folder, diagnostics_df):
    data = []
    labels = []
    metadata = []

    # Check if the folder exists
    if not os.path.exists(ecg_folder):
        raise FileNotFoundError(f"ECG data folder not found: {ecg_folder}")

    for file_name in os.listdir(ecg_folder):
        if file_name.endswith('.csv'):
            # Read ECG data
            ecg_data = pd.read_csv(os.path.join(ecg_folder, file_name), header=None)
            ecg_data = ecg_data.to_numpy()  # Convert to numpy array

            # Retrieve associated label and metadata
            record_id = file_name.replace('.csv', '')
            record_info = diagnostics_df[diagnostics_df['FileName'] == record_id]

            # Skip if no label information is available
            if record_info.empty:
                raise ValueError(f"No diagnostic information found for file: {file_name}")

            # Get rhythm and condition acronyms directly
            rhythm_acronym = record_info['Rhythm'].values[0].strip()

            # The author of the dataset seems like did an oopsie
            if rhythm_acronym == "SA":
                rhythm_acronym = "SI"
                
            # Validate rhythm and condition acronyms
            if rhythm_acronym not in valid_rhythms:
                raise ValueError(f"Unknown rhythm acronym '{rhythm_acronym}' found in file: {file_name}")

            # Set label as rhythm acronym (or use condition acronym if preferred)
            label = rhythm_acronym  # Use rhythm acronym as label

            # Additional metadata
            patient_age = record_info["PatientAge"].values[0]
            ventricular_rate = record_info["VentricularRate"].values[0]
            atrial_rate = record_info["AtrialRate"].values[0]

            data.append(ecg_data)
            labels.append(label)
            metadata.append({
                "patient_age": patient_age,
                "ventricular_rate": ventricular_rate,
                "atrial_rate": atrial_rate
            })

    return data, np.array(labels), metadata


# Load ECG data and labels
ecg_data, ecg_labels, ecg_metadata = load_ecg_data(ecg_folder, diagnostics_df)

# Standardize each ECG signal individually
scaler = StandardScaler()
ecg_data = [scaler.fit_transform(sample) for sample in ecg_data]

# Define a fixed sequence length (pad or truncate to this length)
sequence_length = 5000  # Adjust as per dataset requirements


# Function to pad or truncate each ECG signal to a fixed length
def preprocess_sequence(data, length):
    processed_data = []
    for sample in data:
        if sample.shape[0] > length:
            processed_data.append(sample[:length])
        else:
            pad_width = length - sample.shape[0]
            processed_data.append(np.pad(sample, ((0, pad_width), (0, 0)), mode="constant"))
    return np.array(processed_data)


# initiating
ecg_data = preprocess_sequence(ecg_data, sequence_length)

In [4]:
from collections import Counter

# Filter out classes with fewer than 2 instances
label_counts = Counter(ecg_labels)
print(label_counts)
filtered_indices = [i for i, label in enumerate(ecg_labels) if label_counts[label] > 1]
filtered_ecg_data = ecg_data[filtered_indices]
filtered_ecg_labels = ecg_labels[filtered_indices]

# Train-test split with filtered data
X_train, X_test, y_train, y_test = train_test_split(filtered_ecg_data, filtered_ecg_labels, test_size=0.2,
                                                    random_state=42, stratify=filtered_ecg_labels)

# Verify shapes and data summary
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Training labels shape:", y_train.shape)
print("Testing labels shape:", y_test.shape)
print("Metadata example:", ecg_metadata[0])


Counter({'SB': 3889, 'SR': 1826, 'AFIB': 1780, 'ST': 1568, 'SVT': 587, 'AF': 445, 'SI': 399, 'AT': 121, 'AVNRT': 16, 'AVRT': 8, 'SAAWR': 7})
Training data shape: (8516, 5000, 12)
Testing data shape: (2130, 5000, 12)
Training labels shape: (8516,)
Testing labels shape: (2130,)
Metadata example: {'patient_age': 82, 'ventricular_rate': 143, 'atrial_rate': 144}
